In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# --- 1. Generación de Datos Sintéticos (Simulación) ---
# En un escenario real, estos datos vendrían de una base de datos del banco.

np.random.seed(42) # Para reproducibilidad

num_candidates = 500

data = {
    'años_exp_total': np.random.randint(1, 20, num_candidates),
    'años_exp_banca': np.random.randint(0, 10, num_candidates),
    'años_exp_ventas': np.random.randint(0, 15, num_candidates),
    'nivel_educativo': np.random.choice(['Bachiller', 'Universitario', 'Posgrado'], num_candidates, p=[0.1, 0.6, 0.3]),
    'ha_sido_ceo_logistica': np.random.choice([0, 1], num_candidates, p=[0.85, 0.15]), # 15% CEO logística
    'ha_sido_asesor_comercial': np.random.choice([0, 1], num_candidates, p=[0.7, 0.3]), # 30% Asesor comercial
    'habilidades_liderazgo_score': np.random.randint(1, 10, num_candidates), # Score del 1 al 10
    'habilidades_negociacion_score': np.random.randint(1, 10, num_candidates),
    'ingles_fluido': np.random.choice([0, 1], num_candidates, p=[0.6, 0.4]),
}

df = pd.DataFrame(data)

# Simular la variable 'contratado' (0 o 1) con cierta lógica y ruido
# Una probabilidad más alta si tienen experiencia relevante, educación y habilidades
df['prob_contratado'] = (
    0.05 * df['años_exp_banca'] +
    0.03 * df['años_exp_ventas'] +
    0.02 * df['habilidades_liderazgo_score'] +
    0.02 * df['habilidades_negociacion_score'] +
    0.1 * df['ingles_fluido'] +
    0.05 * df['ha_sido_ceo_logistica'] + # CEO podría ser un plus por liderazgo general
    0.07 * df['ha_sido_asesor_comercial'] + # Asesor comercial es más directo a ventas
    df['nivel_educativo'].apply(lambda x: 0.15 if x == 'Posgrado' else (0.1 if x == 'Universitario' else 0.05)) +
    np.random.normal(0, 0.1, num_candidates) # Ruido aleatorio
)

# Convertir la probabilidad a un resultado binario 'contratado'
df['contratado'] = (df['prob_contratado'] > np.percentile(df['prob_contratado'], 60)).astype(int) # Contratar al 40% superior

# Eliminar la columna de probabilidad intermedia
df = df.drop(columns=['prob_contratado'])

print("--- Datos Sintéticos Generados (Primeras 5 Filas) ---")
print(df.head())
print("\nConteo de Contratados/No Contratados:")
print(df['contratado'].value_counts())

# --- 2. Preprocesamiento de Datos y Preparación para el Modelo ---

# Definir variables numéricas y categóricas
numeric_features = ['años_exp_total', 'años_exp_banca', 'años_exp_ventas',
                    'habilidades_liderazgo_score', 'habilidades_negociacion_score']
categorical_features = ['nivel_educativo', 'ha_sido_ceo_logistica', 'ha_sido_asesor_comercial', 'ingles_fluido']

# Crear un preprocesador usando ColumnTransformer y Pipeline
# Escalado para numéricas, One-Hot Encoding para categóricas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Separar características (X) y variable objetivo (y)
X = df.drop('contratado', axis=1)
y = df['contratado']

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# --- 3. Entrenamiento del Modelo de Regresión Logística ---

# Crear el pipeline: preprocesador + modelo
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', LogisticRegression(solver='liblinear', random_state=42))])

# Entrenar el modelo
model.fit(X_train, y_train)

# --- 4. Evaluación del Modelo (en datos simulados) ---

y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1] # Probabilidad de ser contratado (clase 1)

print("\n--- Evaluación del Modelo en Datos de Prueba (Simulados) ---")
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred))
print(f"ROC AUC Score: {roc_auc_score(y_test, y_prob):.2f}")

# --- 5. Tu Perfil (Como un Nuevo Candidato) ---

# Ingresa tus datos aquí, basados en tu descripción
your_profile = pd.DataFrame([{
    'años_exp_total': 7, # 2023-2021=2 + 2021-2020=1 + 2020-2018=2 + 2018-2016=2 = 7 años
    'años_exp_banca': 0, # Asumiendo que no tienes experiencia directa en banca como sector
    'años_exp_ventas': 5, # Ej. 2 años como RSM + 1 año como CA + 2 años previos si aplica
    'nivel_educativo': 'Posgrado', # Asumiendo que tienes un posgrado
    'ha_sido_ceo_logistica': 1, # Sí, fuiste CEO en River Dispatch
    'ha_sido_asesor_comercial': 1, # Sí, fuiste Asesor Comercial en Deporpas
    'habilidades_liderazgo_score': 9, # Auto-evaluación alta basada en roles gerenciales/directivos
    'habilidades_negociacion_score': 8, # Auto-evaluación alta
    'ingles_fluido': 1 # Asumiendo que sí
}])

print("\n--- Tu Perfil para la Predicción ---")
print(your_profile)

# --- 6. Predicción de Tu Probabilidad de Contratación ---

your_hiring_probability = model.predict_proba(your_profile)[:, 1][0]
your_prediction = model.predict(your_profile)[0]

print(f"\n--- Predicción para Tu Perfil ---")
print(f"Probabilidad de ser contratado en Banco Bolivariano (según este modelo simulado): {your_hiring_probability:.2%}")

if your_prediction == 1:
    print("Según el modelo, hay una ALTA probabilidad de que seas contratado.")
else:
    print("Según el modelo, la probabilidad de contratación es BAJA.")

# --- Interpretación de los Coeficientes (para entender la importancia de las variables) ---
print("\n--- Coeficientes del Modelo (Importancia de las Variables) ---")
# Esto es más complejo con OneHotEncoder y StandardScaler, pero se puede extraer.
# Para simplificar, obtenemos los nombres de las características después del preprocesamiento
feature_names = model.named_steps['preprocessor'].get_feature_names_out()
coefficients = model.named_steps['classifier'].coef_[0]

coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': coefficients})
coef_df['Absolute_Coefficient'] = np.abs(coef_df['Coefficient'])
coef_df = coef_df.sort_values(by='Absolute_Coefficient', ascending=False)

print("Las variables con mayor impacto en la predicción (positiva o negativamente) son:")
print(coef_df.head(10)) # Muestra las 10 variables más influyentes

--- Datos Sintéticos Generados (Primeras 5 Filas) ---
   años_exp_total  años_exp_banca  años_exp_ventas nivel_educativo  \
0               7               5                0       Bachiller   
1              15               2               12        Posgrado   
2              11               5               14        Posgrado   
3               8               6               10        Posgrado   
4               7               9                5        Posgrado   

   ha_sido_ceo_logistica  ha_sido_asesor_comercial  \
0                      0                         1   
1                      0                         0   
2                      0                         0   
3                      0                         0   
4                      0                         1   

   habilidades_liderazgo_score  habilidades_negociacion_score  ingles_fluido  \
0                            2                              4              0   
1                            9          